In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as tik
import pandas as pd
from datetime import datetime, timezone, timedelta

from scipy.stats import norm
import matplotlib.mlab as mlab

from operator import add

#from matplotlib import gridspec
#from scipy.optimize import curve_fit

# Input data and log

In [ ]:
#file path and name
filepath = './rawdata/'
filename = 'Feb17_initMatrix.dat'

time_category, error, direction, timercount, reference = np.loadtxt(filepath + filename, comments='#', unpack=True)

In [ ]:
logpath = './log/'
logfilename = 'PMX_2021-02-17.dat'

df = pd.read_csv(logpath + logfilename, delim_whitespace=True, comment='#', skiprows=416)

In [ ]:
df

In [ ]:
iteration = 100

list_amp = [1.5, 1.8, 2.1, 2.4, 2.7, 3.0]
list_intv = ['0.400', '0.800', '1.200', '1.600', '2.000', '2.400']

matrix_size = 36

pack_size = 1

In [ ]:
min(error)

## Set variables and Check some charactaristics
* function is available only for data range

In [ ]:
time = timercount/200e6

skip = 50
flag_threshold = 4

casp_threshold = -100
edge_coverage = 40

Deg = 360/52000

In [ ]:
storepath = './plots/'
storename = 'Nov30_AmpTime'

In [ ]:
#first scketch of the rawdata
fig01, axarr01 = plt.subplots(1,2,figsize=(14,7))
ax01 = axarr01.ravel()

ax01[0].plot(time[::100], reference[::100])
ax01[0].set_xlabel('time[sec]')
ax01[0].set_ylabel('reference count')

ax01[1].scatter(np.diff(timercount), np.diff(reference))
ax01[1].set_xlabel('difference time of lines')
ax01[1].set_ylabel('difference counts of lines')

In [ ]:
#fig01.savefig(storepath + storename + '_RawAndCorr.png')

In [ ]:
def switching_time(dataframe, list_ampere, list_interval):
    JST = timezone(timedelta(hours=+9), 'JST')
    initials = []
    list_UNIXTIME = []
    
    for i in range(iteration):
        list_UNIXTIME.append([])
    
    for i in list_ampere:
        initials.append([])
        for j in list_interval:
            list_datetime = ((dataframe[dataframe.iloc[:,5] == i])[(dataframe[dataframe.iloc[:,5] == i]).iloc[:,8] == j]).iloc[:,0:2].to_numpy()
            for k in range(len(list_datetime)):
                if k%pack_size == 0:
                    list_UNIXTIME[int(k/pack_size)] = (((datetime.strptime(list_datetime[k][0]+' '+list_datetime[k][1],"%Y-%m-%d %H:%M:%S-%Z")).replace(tzinfo=timezone.utc)).astimezone(JST)).timestamp()
                    pass
                pass
            initials[list_ampere.index(i)].append([])
            initials[list_ampere.index(i)][list_interval.index(j)] = min(list_UNIXTIME)
            pass
        pass
    
    log_end = dataframe[dataframe.iloc[:,2] == 'OFF'].iloc[-1,:]
    switching_end = (((datetime.strptime(log_end[0]+' '+log_end[1],"%Y-%m-%d %H:%M:%S-%Z")).replace(tzinfo=timezone.utc)).astimezone(JST)).timestamp()
    
    return initials, switching_end

In [ ]:
sw_init, sw_end = switching_time(df, list_amp, list_intv)

In [ ]:
def switching_dataline(switching_initials, switching_end):
    initial_lines = []
    final_lines = []

    for i in range(len(list_amp)):
        for j in range(len(list_intv)-1):
            initial_lines.append(min(np.where((switching_initials[i][j] <= time_category)&(time_category < switching_initials[i][j+1]))[0]))
            final_lines.append(max(np.where((switching_initials[i][j] <= time_category)&(time_category < switching_initials[i][j+1]))[0]))
            
            pass
        if i != len(list_amp)-1:
            initial_lines.append(min(np.where((switching_initials[i][len(list_intv)-1] <= time_category)&(time_category < switching_initials[i+1][0]))[0]))
            final_lines.append(max(np.where((switching_initials[i][len(list_intv)-1] <= time_category)&(time_category < switching_initials[i+1][0]))[0]))
            pass
        else:
            initial_lines.append(min(np.where((switching_initials[i][len(list_intv)-1] <= time_category)&(time_category < switching_end))[0]))
            final_lines.append(max(np.where((switching_initials[i][len(list_intv)-1] <= time_category)&(time_category < switching_end))[0]))
            pass
        pass
    
    return initial_lines, final_lines

In [ ]:
ilines, flines = switching_dataline(sw_init, sw_end)

In [ ]:
time_block = []
reference_block = []

for i in range(len(ilines)):
    time_block.append(time[ilines[i]:flines[i]])
    reference_block.append(reference[ilines[i]:flines[i]])

In [ ]:
fig02 = plt.figure(figsize=(10,7))
for i in range(matrix_size):
    plt.plot(time_block[i], reference_block[i])
    pass

In [ ]:
def edge_detection(specific_time, specific_refcount):    
    linear_refcount = np.array(np.zeros(len(specific_refcount)))
    
    count_offset = 0
    casp_flag = 0
    casp = np.where(np.diff(specific_refcount) <= casp_threshold)[0].tolist()
    
    for i in range(len(specific_refcount)):
        linear_refcount[i] = specific_refcount[i]+count_offset
        if casp:
            if i == casp[casp_flag]:
                count_offset += specific_refcount[casp[casp_flag]]
                if len(casp) > casp_flag+1:
                    casp_flag += 1
                    pass
                pass
            pass
        pass
    
    modLinear = linear_refcount[::skip]
    modTime = specific_time[::skip]
    
    positive = np.diff(modLinear) > 0.
    edge_initial = np.array([])
    edge_initial = np.where(positive[1:] & ~positive[:-1])[0]
    edge_final = np.array([])
    rep_edge = np.where(~positive[1:] & positive[:-1])[0]
    for i in rep_edge:
        edge_flag = 0
        for j in range(edge_coverage):
            if (np.diff(modLinear))[i+j] == 0:
                edge_flag += 1
                if edge_flag == flag_threshold:
                    edge_final = np.append(edge_final, i+j+flag_threshold)
                    break
                    pass
                pass
            pass
 
    return modTime, modLinear, edge_initial, edge_final

In [ ]:
def stable_detection(specific_time, specific_refcount):    
    linear_refcount = np.array(np.zeros(len(specific_refcount)))
    
    count_offset = 0
    casp_flag = 0
    casp = np.where(np.diff(specific_refcount) <= casp_threshold)[0]

    for i in range(len(specific_refcount)):
        linear_refcount[i] = specific_refcount[i]+count_offset
        if i == casp[casp_flag]:
            count_offset += specific_refcount[casp[casp_flag]]
            if len(casp) > casp_flag+1:
                casp_flag += 1
                pass
            pass
        pass

    modLinear = linear_refcount[::skip]
    modTime = specific_time[::skip]

    stable_position = np.array([])
    positive = (np.diff(modLinear) > 0.)
    rep_stable = np.where(positive[1:] & ~positive[:-1])[0]

    for i in range(len(rep_stable)):
        #if i%pack_size == 0:
            edge_flag = 0
            for j in range(flag_threshold):
                if (np.diff(modLinear))[rep_stable[i]-j] == 0:
                    edge_flag += 1
                    if edge_flag == flag_threshold:
                        stable_position = np.append(stable_position, rep_stable[i]-60)
                        break
                        pass

                    pass
                pass
            pass
 
    return modTime, modLinear, stable_position

In [ ]:
#make variables
linTime = []
linRef = []
edge_initial = []
edge_final = []

proceeded_degrees = []
operating_time = []


for i in range(matrix_size):
    linTime.append([])
    linRef.append([])
    edge_initial.append([])
    edge_final.append([])
    linTime[i], linRef[i], edge_initial[i], edge_final[i] = edge_detection(time_block[i],reference_block[i])
    
    proceeded_degrees.append([])
    operating_time.append([])
    proceeded_degrees[i] = np.append(proceeded_degrees[i], np.zeros(len(edge_initial[i])))
    operating_time[i] = np.append(operating_time[i], np.zeros(len(edge_initial[i])))
    for j in range(min(len(edge_initial[i]),len(edge_final[i]))):
        proceeded_degrees[i][j] = (linRef[i][int(edge_final[i][j])]-linRef[i][int(edge_initial[i][j])])*Deg
        operating_time[i][j] = linTime[i][int(edge_final[i][j])]-linTime[i][int(edge_initial[i][j])]

In [ ]:
error_pack = [[0]]

In [ ]:
error_pack = []
for i in range(matrix_size):
    error_pack.append([i])
    pass
error_pack

In [ ]:
#avoid error
error_position = []
error_flag0 = 0

error_flag2 = 0

for i in range(matrix_size):
    error_flag1 = 1
    error_position.append([])
    for j in range(100):
        if i == error_pack[error_flag0][0]:
            if error_flag1 <= len(error_pack[error_flag0])-1:
                if j == error_pack[error_flag0][error_flag1]:
                    #print(f'{error_pack[error_flag0][0]} {error_pack[error_flag0][error_flag1]}')
                    error_position[i].append([])
                    error_position[i][j] = False
                    error_flag1 += 1
                    error_flag2 = 1
                    pass
                else:
                    error_position[i].append([])
                    error_position[i][j] = True
                    pass
                pass
            else:
                error_position[i].append([])
                error_position[i][j] = True
                pass
            pass
        else:
            error_position[i].append([])
            error_position[i][j] = True
            pass
        pass
    if error_flag2 == 1:
        if error_flag0 != len(error_pack)-1:
            error_flag0 += 1
            error_flag2 = 0
            pass
        pass
    pass

In [ ]:
np.where(np.array(error_position) == False)[0]

In [ ]:
len(error_position)

In [ ]:
for i in range(len(error_position)):
    print(f'{i} {len(error_position[i])} {len(operating_time[i])}')

In [ ]:
opeTime_woErr = []
proDeg_woErr = []
linInitRef_woErr = []
linFinRef_woErr = []
linInitTime_woErr = []
linFinTime_woErr = []
for i in range(matrix_size):
    opeTime_woErr.append([])
    proDeg_woErr.append([])
    linInitRef_woErr.append([])
    linFinRef_woErr.append([])
    linInitTime_woErr.append([])
    linFinTime_woErr.append([])
    for j in range(len(error_position[i])):
        if error_position[i][j] != False:
            opeTime_woErr[i].append(operating_time[i][j])
            proDeg_woErr[i].append(proceeded_degrees[i][j])
            linInitRef_woErr[i].append((linRef[i][edge_initial[i][j]]%52000)*360/52000)
            linFinRef_woErr[i].append((linRef[i][int(edge_final[i][j])]%52000)*360/52000)
            linInitTime_woErr[i].append(linTime[i][int(edge_initial[i][j])])
            linFinTime_woErr[i].append(linTime[i][int(edge_final[i][j])])

In [ ]:
len(edge_final[0])

In [ ]:
for i in range(10):
    print(f'{i} {edge_initial[0][i]} {edge_final[0][i]}')

In [ ]:
len(error_position)

In [ ]:
len(error_position[0])

## Set variables2

In [ ]:
time = timercount/200e6

plot_slice = 50
capture_slice = 500

pos_roll = 5

casp_threshold = -100

Deg = 360/52000

In [ ]:
storepath = './plots/'
storename = 'Feb17_AmpTime'

In [ ]:
#first scketch of the rawdata
fig01, axarr01 = plt.subplots(1,2,figsize=(14,7))
ax01 = axarr01.ravel()

ax01[0].plot(time[::100], reference[::100])
ax01[0].set_xlabel('time[sec]')
ax01[0].set_ylabel('reference count')

ax01[1].scatter(np.diff(timercount), np.diff(reference))
ax01[1].set_xlabel('difference time of lines')
ax01[1].set_ylabel('difference counts of lines')

In [ ]:
fig01.savefig(storepath + storename + '_RawAndCorr.png')

In [ ]:
def packet_capture(dataframe, list_ampere, list_interval, UnixTime_Data):
    JST = timezone(timedelta(hours=+9), 'JST')
    initial_UnixTime = []
    switching_UnixTime = []
    start_UnixTime = []

    for i in range(iteration):
        start_UnixTime.append([])

    for i in list_ampere:
        for j in list_interval:
            list_datetime = ((dataframe[dataframe.iloc[:,5] == i])[(dataframe[dataframe.iloc[:,5] == i]).iloc[:,8] == j]).iloc[:,0:2].to_numpy()
            for k in range(len(list_datetime)):
                if k%pack_size == 0:
                    start_UnixTime[int(k/pack_size)] = (((datetime.strptime(list_datetime[k][0]+' '+list_datetime[k][1],"%Y-%m-%d %H:%M:%S-%Z")).replace(tzinfo=timezone.utc)).astimezone(JST)).timestamp()
                    initial_UnixTime = np.append(initial_UnixTime, start_UnixTime[int(k/pack_size)])
                    pass
                pass
            switching_UnixTime.append(min(start_UnixTime))
            pass
        pass

    stop_UnixTime = dataframe[dataframe.iloc[:,2] == 'OFF'].iloc[-1,:]
    end_UnixTime = (((datetime.strptime(stop_UnixTime[0]+' '+stop_UnixTime[1],"%Y-%m-%d %H:%M:%S-%Z")).replace(tzinfo=timezone.utc)).astimezone(JST)).timestamp()

    captured_initials = []
    captured_switching = []
    
    for i in range(len(switching_UnixTime)):
        captured_switching.append(max(np.where(time_category[::capture_slice] <= switching_UnixTime[i])[0]))
        pass
    
    for i in range(len(initial_UnixTime)):
        captured_initials.append(max(np.where(time_category[::capture_slice] <= initial_UnixTime[i])[0]))
        pass
    
    captured_end = min(np.where(end_UnixTime + pos_roll <= time_category[::capture_slice])[0])
    
    return captured_switching, captured_initials, captured_end

In [ ]:
swt, inits, end = packet_capture(df, list_amp, list_intv, time_category)

In [ ]:
#block
time_block = []
reference_block = []

for i in range(len(swt)-1):
    time_block.append(time[int(np.where(time == (time[::capture_slice])[swt[i]-pos_roll])[0]):int(np.where(time == (time[::capture_slice])[swt[i+1]-pos_roll])[0])])
    reference_block.append(reference[int(np.where(time == (time[::capture_slice])[swt[i]-pos_roll])[0]):int(np.where(time == (time[::capture_slice])[swt[i+1]-pos_roll])[0])])
    pass
time_block.append(time[int(np.where(time == (time[::capture_slice])[swt[-1]-pos_roll])[0]):int(np.where(time == (time[::capture_slice])[end+pos_roll])[0])])
reference_block.append(reference[int(np.where(time == (time[::capture_slice])[swt[-1]-pos_roll])[0]):int(np.where(time == (time[::capture_slice])[end+pos_roll])[0])])

lincount_block = []
block_initials = []

for i in range(matrix_size):
    lincount_block.append([])
    lincount_block[i] = np.array(np.zeros(len(reference_block[i])))
    
    count_offset = 0
    casp_flag = 0
    casp = np.where(np.diff(reference_block[i]) <= casp_threshold)[0].tolist()

    for j in range(len(reference_block[i])):
        lincount_block[i][j] = reference_block[i][j]+count_offset
        if casp:
            if j == casp[casp_flag]:
                count_offset += reference_block[i][casp[casp_flag]]
                if len(casp) > casp_flag+1:
                    casp_flag += 1
                    pass
                pass
            pass
        pass
    
    block_initials.append([])
    for j in range(iteration):
        block_initials[i].append(int(np.where(time_block[i] == (time[::capture_slice])[(inits[i*100:(i+1)*100])[j]-pos_roll])[0]))
        pass

In [ ]:
print("{}{: >25}{}{: >10}{}".format('|','Variable Name','|','Memory','|'))
print(" ------------------------------------ ")
for var_name in dir():
    if not var_name.startswith("_") and sys.getsizeof(eval(var_name)) > 1000: #ここだけアレンジ
        print("{}{: >25}{}{: >10}{}".format('|',var_name,'|',sys.getsizeof(eval(var_name)),'|'))

In [ ]:
del df

In [ ]:
error_pack = [[1,83], [4,50], [5,61,82], [6,55], [7,38], [9,20,81], [10,4,17,57,79],
             [11,30], [16,33,64,86], [17,20,91], [19,71], [21,2,63], [22,40],
             [23,81], [25,42,74], [26,80], [28,38,51,91], [29,58], [31,18], [33,21,91],
             [34,62,84], [35,18,35,39,85]]

In [ ]:
#avoid error
error_position = []
error_flag0 = 0

error_flag2 = 0

for i in range(matrix_size):
    error_flag1 = 1
    error_position.append([])
    for j in range(100):
        if i == error_pack[error_flag0][0]:
            if error_flag1 <= len(error_pack[error_flag0])-1:
                if j == error_pack[error_flag0][error_flag1]:
                    #print(f'{error_pack[error_flag0][0]} {error_pack[error_flag0][error_flag1]}')
                    error_position[i].append([])
                    error_position[i][j] = False
                    error_flag1 += 1
                    error_flag2 = 1
                    pass
                else:
                    error_position[i].append([])
                    error_position[i][j] = True
                    pass
                pass
            else:
                error_position[i].append([])
                error_position[i][j] = True
                pass
            pass
        else:
            error_position[i].append([])
            error_position[i][j] = True
            pass
        pass
    if error_flag2 == 1:
        if error_flag0 != len(error_pack)-1:
            error_flag0 += 1
            error_flag2 = 0
            pass
        pass
    pass

In [ ]:
onecycle_time = []
proceeded_degrees = []
for i in range(matrix_size):
    onecycle_time.append([])
    proceeded_degrees.append([])
    for j in range(iteration):
        if j != iteration - 1:
            hoge = (time_block[i][block_initials[i][j]:block_initials[i][j+1]])-time_block[i][block_initials[i][j]]
            place = min(np.where(hoge >= cut_place[i%6])[0])
            onecycle_time[i].append(time_block[i][block_initials[i][j] + place]-time_block[i][block_initials[i][j]])
            proceeded_degrees[i].append((lincount_block[i][block_initials[i][j] + place]-lincount_block[i][block_initials[i][j]])*Deg)
            pass
        else:
            hoge = (time_block[i][block_initials[i][j]:-1])-time_block[i][block_initials[i][j]]
            place = min(np.where(hoge >= cut_place[i%6])[0])
            onecycle_time[i].append(time_block[i][block_initials[i][j] + place]-time_block[i][block_initials[i][j]])
            proceeded_degrees[i].append((lincount_block[i][block_initials[i][j] + place]-lincount_block[i][block_initials[i][j]])*Deg)
        pass
    pass

In [ ]:
proDeg_woErr = []
linCount_woErr = []
for i in range(matrix_size):
    proDeg_woErr.append([])
    linCount_woErr.append([])
    for j in range(iteration):
        if error_position[i][j] != False:
            proDeg_woErr[i].append(proceeded_degrees[i][j])
            linCount_woErr[i].append((lincount_block[i][block_initials[i]])[j])

In [ ]:
#region
figX = plt.figure(figsize=(22,12))
for i in range(matrix_size):
    plt.plot(time_block[i], reference_block[i])
    pass
for i in range(matrix_size):
    if i%6 == 5:
        plt.axvline(time_block[i][-1], color='b', label=f'end_{round((1.5+0.3*int(i/6)),2)}sec')

In [ ]:
cut_place = [2.5, 3.0, 3.5, 3.9, 4.33, 4.8]

In [ ]:
i = 35
for j in range(99):
    if error_position[i][j] != False:
        hoge = (time_block[i][block_initials[i][j]:block_initials[i][j+1]])-time_block[i][block_initials[i][j]]
        place = min(np.where(hoge >= cut_place[i%6])[0])
        plt.plot(((time_block[i][block_initials[i][j]:block_initials[i][j] + place])[::plot_slice])[:-1]-time_block[i][block_initials[i][j]],
                        np.diff((lincount_block[i][block_initials[i][j]:block_initials[i][j] + place])[::plot_slice]), label=f'num_{j}')
    #plt.legend(fontsize=20)

In [ ]:
ref_threshold = 1000

cut_sum = []
for j in range(99):
    hoge = np.where(np.diff((lincount_block[i][block_initials[i][j]:block_initials[i][j] + place])[::plot_slice]) > ref_threshold)[0]
    if hoge:
        cut_sum.append(j)
        pass
cut_sum
                

In [ ]:
fig10, axarr10 = plt.subplots(6,6,figsize=(24,24))
ax10 = axarr10.ravel()

for i in range(matrix_size):
    for j in range(iteration-1):
        if error_position[i][j] != False:
            hoge = (time_block[i][block_initials[i][j]:block_initials[i][j+1]])-time_block[i][block_initials[i][j]]
            place = min(np.where(hoge >= cut_place[i%6])[0])
            ax10[i].plot(((time_block[i][block_initials[i][j]:block_initials[i][j] + place])[::plot_slice])[:-1]-time_block[i][block_initials[i][j]],
                        np.diff((lincount_block[i][block_initials[i][j]:block_initials[i][j] + place])[::plot_slice]))
            ax10[i].set_title(f'{round((1.5+0.3*int(i/6)),2)}A_{0.5*(1+i%6)}sec')
        

In [ ]:
fig10.savefig(storepath + storename +'_speed1-avoidERROR.png', facecolor='white')

In [ ]:
i = 14
j = 20
plt.plot(((time_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice])[:-1]-time_block[i][block_initials[i][j]],
        np.diff((lincount_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice]))

In [ ]:
i = 19
j = 20

hoge = np.diff((lincount_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice]) > 0
fuga = np.where(~hoge[:-1] & hoge[1:])
piyo = ((time_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice])[1:]
kue = (((time_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice])[1:])[fuga]

figY = plt.figure(figsize=(10,7))

plt.plot(piyo-kue, np.diff((lincount_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice]), color='k', label='new black belt')
plt.axvline(0.95)
plt.axvline(1.15)
plt.xlabel('time[sec]')
plt.ylabel('difference of counts (~speed)')
plt.xlim([-0.5,2.])
plt.ylim([-20,330])
plt.legend(loc='upper left', fontsize=15)

In [ ]:
proceeded_degrees[i][j]

In [ ]:
figY.savefig(storepath + storename + '_form_21e-1A-1sec_num20.png', transparent=True)

In [ ]:
i = 19
j = 20

toki = np.diff(((time_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice]))
basho = np.diff((lincount_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice])

hoge = np.diff((lincount_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice]) > 0
fuga = np.where(~hoge[:-1] & hoge[1:])
piyo = ((time_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice])[:-1]
kue = (((time_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice])[:-1])[fuga]
saku = basho/toki

figY = plt.figure(figsize=(10,7))

plt.plot((piyo-kue)[1:], np.diff(saku)/np.diff(toki), color='k', label='new black belt')
plt.axvline(0)
plt.xlabel('time[sec]')
plt.ylabel('acceleration')
plt.xlim([0.95,1.15])
plt.ylim([-20,1e9])
plt.legend(loc='upper left', fontsize=15)

* <span style="font-size: 200%; color: red;">Here is a check point of analysis.</span>

In [ ]:
fig11, axarr11 = plt.subplots(6,6,figsize=(24,24))
ax11 = axarr11.ravel()

for i in range(matrix_size):
    ax11[i].scatter(np.array(linCount_woErr[i])%52000*Deg,proDeg_woErr[i])
    ax11[i].set_title(f'{round((1.5+0.3*int(i/6)),2)}A_{round(0.4 + 0.4*i%6,2)}sec')

In [ ]:
fig11.savefig(storepath + storename + '_POSvsDEG-avoidERROR.png',facecolor='white')

In [ ]:
fig12, axarr12 = plt.subplots(6,6,figsize=(24,24))
ax12 = axarr12.ravel()

for i in range(matrix_size):
    ax12[i].hist(proDeg_woErr[i])
    ax12[i].set_title(f'{round((1.5+0.3*int(i/6)),2)}A_{round(0.4 + 0.4*i%6,2)}sec')

In [ ]:
fig12.savefig(storepath + storename + '_histDEG-avoidERROR.png', facecolor='white')

In [ ]:
proDeg_mean = []
proDeg_stdv = []

for i in range(6):
    proDeg_mean.append([])  
    proDeg_stdv.append([])
for i in range(matrix_size):
    proDeg_mean[int(i/6)].append(np.mean(proDeg_woErr[i]))
    proDeg_stdv[int(i/6)].append(np.std(proDeg_woErr[i]))
    pass

In [ ]:
figXX = plt.figure(figsize=(10,7))
plt.rcParams["font.size"] = 20
for i in range(6):
    plt.plot(proDeg_mean[i], proDeg_stdv[i], label=f'{round(1.5+i*0.3,2)}A')
    plt.scatter(proDeg_mean[i], proDeg_stdv[i])
    pass
plt.xlabel('respective mean of proceeded degrees[degree]')
plt.ylabel('standard deviation[degree]')
plt.xlim([-30,360])
plt.ylim([0,10.5])
# plt.axvline(360)
plt.legend(loc='lower right', fontsize=20)

In [ ]:
figXX.savefig(storepath + storename + '_DEGvsSTD.png', transparent=True)

In [ ]:
print("{}{: >25}{}{: >10}{}".format('|','Variable Name','|','Memory','|'))
print(" ------------------------------------ ")
for var_name in dir():
    if not var_name.startswith("_") and sys.getsizeof(eval(var_name)) > 1000: #ここだけアレンジ
        print("{}{: >25}{}{: >10}{}".format('|',var_name,'|',sys.getsizeof(eval(var_name)),'|'))

In [ ]:
#del df, initial_lines, time, rep, hoge

# Plots

In [ ]:
fig10 ,axarr10 = plt.subplots(6,6,figsize=(26,28))
ax10 = axarr10.ravel()

for i in range(matrix_size):
    for j in range(min(len(edge_initial[i]),len(edge_final[i]))):
        #if error_position[i][j] != False:
        ax10[i].plot((linTime[i][:-1])[int(edge_initial[i][j]):int(edge_final[i][j])]-(linTime[i][:-1])[int(edge_initial[i][j])],
                    (np.diff(linRef[i]))[int(edge_initial[i][j]):int(edge_final[i][j])])
        ax10[i].set_title(f'{round((1.2+0.3*int(i/6)),2)}A_{0.5*(1+i%6)}sec')
        #ax10[i].xaxis.set_major_locator(tik.MultipleLocator(0.5))

In [ ]:
#fig10.savefig(storepath + storename +'_speed1.png', facecolor='white')

In [ ]:
i = 0

fig = plt.figure(figsize=(10,7))
#for j in range(min(len(edge_initial[i]),len(edge_final[i]))):
for j in range(0,10):
    #if error_position[i][j] != False:
    plt.plot((linTime[i][:-1])[int(edge_initial[i][j]):int(edge_final[i][j])]-(linTime[i][:-1])[int(edge_initial[i][j])],
                (np.diff(linRef[i]))[int(edge_initial[i][j]):int(edge_final[i][j])])
    plt.title(f'{round((1.2+0.3*int(i/6)),2)}A_{0.5*(1+i%6)}sec')

In [ ]:
fig11 ,axarr11 = plt.subplots(6,6,figsize=(24,22))
ax11 = axarr11.ravel()

for i in range(matrix_size):
    ax11[i].scatter(linInitRef_woErr[i],proDeg_woErr[i], s=20)
    ax11[i].set_title(f'{round((1.2+0.3*int(i/6)),2)}A_{0.5*(1+i%6)}sec')
    pass

In [ ]:
#fig11.savefig(storepath + storename + '_TvsDEG.png', facecolor='white')

In [ ]:
fig12 = plt.figure(figsize=(10,7))
for i in range(matrix_size):
    plt.scatter(linInitRef_woErr[i], proDeg_woErr[i])
    pass
plt.title('2.1A 0.55sec position VS degrees')
plt.xlabel('position[deg]')
plt.ylabel('proceeded degrees[deg]')

In [ ]:
int(len(linInitRef_woErr[0])/16)

In [ ]:
hoge = []
for i in range(int(len(linInitRef_woErr[0])/16)):
    hoge.append(linInitRef_woErr[0][16*i:16*(i+1)])
    pass
#hoge

In [ ]:
360/23

In [ ]:
plt.hist(linInitRef_woErr[0],range=[0,360],bins=15)

In [ ]:
hist_color = ['bisque', 'lightgreen', 'skyblue']
fuga = []
for i in range(int(len(linInitRef_woErr[0])/16)):
    fuga.append(hist_color[i%3])

In [ ]:
fig = plt.figure(figsize=(10,7))
plt.hist(hoge,range=[0,360],bins=15,stacked=True,color=fuga)

In [ ]:
#fig12.savefig(storepath + storename + '_POSvsDEG.png', facecolor='white')

In [ ]:
fig13 = plt.figure()
for i in range(matrix_size):
    plt.hist(proDeg_woErr[i], bins=15)
    pass
plt.title('2.1A 0.55sec proceeded degrees hist')
plt.xlabel('proceeded degrees[deg]')
plt.ylabel('number of events')

In [ ]:
#fig13.savefig(storepath + storename + '_histDEG.png', facecolor='white')

In [ ]:
#Fitting histogram
fig14 = plt.figure(figsize=(10,7))
mu, std = norm.fit(proDeg_woErr)

# Plot the histogram.
plt.hist(proDeg_woErr, bins=15, density=True, alpha=0.3, color='g', label='2.1A 0.55sec')
plt.legend(fontsize=15,loc='upper right')

# Plot the PDF.
xmin, xmax = plt.xlim([0,40])
plt.ylim([0.,0.18])
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
plt.plot(x, p, 'k', linewidth=2)
plt.title(f'Fit Result: mu={round(mu,2)}, std={round(std,2)}')
plt.xlabel('proceeded degrees[deg]')
plt.ylabel('number of events')

In [ ]:
#fig14.savefig(storepath + storename + '_histDEG_FIT0.png', facecolor='white')

# Others